In [1]:
pip install peft transformers datasets torch scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import get_scheduler
from datasets import load_dataset
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score

# Import PEFT libraries
from peft import LoraConfig, get_peft_model, TaskType


In [7]:

# Simple function to fine-tune a transformer model on a small dataset using LoRA
def fine_tune_transformer_with_lora():
    print("Starting the LoRA fine-tuning process...")

    # Step 1: Load a small dataset for demonstration
    print("Loading dataset...")
    dataset = load_dataset("tweet_eval", "emotion")

    # Use a larger training subset and all available validation samples
    train_dataset = dataset["train"]
    eval_dataset = dataset["validation"]

    # Step 2: Load a small pre-trained model and tokenizer
    print("Loading pre-trained model and tokenizer...")
    model_name = "distilbert-base-uncased"  # Small model, good for CPU
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Step 3: Prepare the data
    def preprocess_data(examples):
        # Convert text to tokens with simple padding
        return tokenizer(
            examples["text"],
            padding="max_length",
            truncation=True,
            max_length=64  # Shorter sequences for faster processing
        )

    print("Preprocessing data...")
    train_encoded = train_dataset.map(preprocess_data, batched=True)
    eval_encoded = eval_dataset.map(preprocess_data, batched=True)

    # Keep only the columns needed for training
    # Ensure 'label' column is renamed to 'labels' if it's not already for the model
    train_encoded = train_encoded.remove_columns(["text"])
    train_encoded = train_encoded.rename_column("label", "labels") # Expected by model
    eval_encoded = eval_encoded.remove_columns(["text"])
    eval_encoded = eval_encoded.rename_column("label", "labels")   # Expected by model


    # Format data for PyTorch
    train_encoded.set_format("torch")
    eval_encoded.set_format("torch")

    # Step 4: Set up data loaders with a slightly larger batch size
    batch_size = 16
    train_loader = DataLoader(train_encoded, shuffle=True, batch_size=batch_size)
    eval_loader = DataLoader(eval_encoded, batch_size=batch_size)

    # Step 5: Initialize the base model
    num_labels = len(set(train_dataset["label"]))  # Number of emotion classes
    base_model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=num_labels
    )

    # Step 5.1: Configure LoRA
    print("Configuring LoRA...")
    lora_config = LoraConfig(
        task_type=TaskType.SEQ_CLS, # Sequence Classification
        r=8,  # Rank of the LoRA matrices (a common value, can be tuned)
        lora_alpha=16, # Alpha scaling factor (often 2*r)
        target_modules=["q_lin", "v_lin"], # Apply LoRA to query and value layers of attention
                                           # For DistilBERT. For BERT, it might be ["query", "value"]
                                           # You can inspect model.named_modules() to find appropriate layers
        lora_dropout=0.1,
        bias="none" # Or "all" or "lora_only"
    )

    # Step 5.2: Wrap the base model with LoRA
    model = get_peft_model(base_model, lora_config)
    print("LoRA model created.")
    model.print_trainable_parameters() # See how many parameters are trainable

    # Step 6: Set up training with adjusted parameters
    # The optimizer will now only update the LoRA parameters and the classifier head
    optimizer = AdamW(model.parameters(), lr=2e-4) # LoRA often benefits from slightly higher LR
    num_epochs = 10  # 3 epochs for better convergence

    # Simple linear learning rate scheduler
    num_training_steps = num_epochs * len(train_loader)
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps
    )

    # Step 7: Simple training loop
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    model.to(device)

    print(f"Training for {num_epochs} epochs...")

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0

        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            # Move batch to device
            batch = {k: v.to(device) for k, v in batch.items()}

            # Forward pass
            outputs = model(**batch) # 'labels' is already in the batch due to rename
            loss = outputs.loss
            train_loss += loss.item()

            # Backward pass
            loss.backward()

            # Update weights
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()

        # Print average loss for the epoch
        avg_train_loss = train_loss / len(train_loader)
        print(f"Average training loss: {avg_train_loss:.4f}")

        # Evaluation phase after each epoch
        model.eval()
        all_predictions = []
        all_true_labels = [] # Renamed to avoid conflict with 'labels' in batch

        for batch in tqdm(eval_loader, desc="Evaluating"):
            batch_labels = batch.pop('labels').to(device) # Extract labels separately
            batch = {k: v.to(device) for k, v in batch.items()}


            with torch.no_grad():
                outputs = model(**batch) # Pass inputs without labels

            predictions = torch.argmax(outputs.logits, dim=-1)
            all_predictions.extend(predictions.cpu().numpy())
            all_true_labels.extend(batch_labels.cpu().numpy())


        # Calculate and print accuracy
        accuracy = accuracy_score(all_true_labels, all_predictions)
        print(f"Validation accuracy: {accuracy:.4f}")

    # Step 8: Inference with the fine-tuned LoRA model
    print("Performing inference with the fine-tuned LoRA model...")

    # Define the emotion labels (based on tweet_eval emotion dataset)
    emotion_labels = {0: "anger", 1: "joy", 2: "optimism", 3: "sadness"}

    # Function to predict emotion for a given text
    def predict_emotion(text):
        model.eval() # Ensure model is in evaluation mode
        # Tokenize the input text
        inputs = tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=64,
            return_tensors="pt"  # Return PyTorch tensors
        )

        # Move inputs to the device
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # Make prediction
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            predicted_class = torch.argmax(logits, dim=-1).item()

        # Return the predicted emotion
        return emotion_labels[predicted_class]

    # Test with example inputs
    test_texts = [
        "I am so happy today, life is great!",
        "This is the worst day ever, everything went wrong.",
        "I'm feeling hopeful about the future.",
        "Why does everything always go wrong for me?"
    ]

    for text in test_texts:
        predicted_emotion = predict_emotion(text)
        print(f"Text: {text}")
        print(f"Predicted Emotion: {predicted_emotion}\n")

    print("LoRA Fine-tuning and inference completed!")

    # To save only the LoRA adapter (very small file):
    # model.save_pretrained("my_lora_adapter")
    # To load it later:
    # from peft import PeftModel
    # loaded_base_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
    # loaded_lora_model = PeftModel.from_pretrained(loaded_base_model, "my_lora_adapter")
    # loaded_lora_model.to(device)
    # print("LoRA adapter loaded and model ready for inference.")

    return model, tokenizer

# Run the fine-tuning process
if __name__ == "__main__":
    fine_tune_transformer_with_lora()

Starting the LoRA fine-tuning process...
Loading dataset...
Loading pre-trained model and tokenizer...
Preprocessing data...


Map:   0%|          | 0/374 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Configuring LoRA...
LoRA model created.
trainable params: 741,124 || all params: 67,697,672 || trainable%: 1.0948
Using device: cuda
Training for 10 epochs...


Epoch 1/10:   0%|          | 0/204 [00:00<?, ?it/s]

...

Validation accuracy: 0.8075


Epoch 10/10:   0%|          | 0/204 [00:00<?, ?it/s]

Average training loss: 0.3047


Evaluating:   0%|          | 0/24 [00:00<?, ?it/s]

Validation accuracy: 0.8021
Performing inference with the fine-tuned LoRA model...
Text: I am so happy today, life is great!
Predicted Emotion: joy

Text: This is the worst day ever, everything went wrong.
Predicted Emotion: anger

Text: I'm feeling hopeful about the future.
Predicted Emotion: optimism

Text: Why does everything always go wrong for me?
Predicted Emotion: sadness

LoRA Fine-tuning and inference completed!
